In [14]:
import numpy as np
import matplotlib.pyplot as plt
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms



In [15]:
transform  = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./data', train =True,transform=transform, download=True)

test_data = torchvision.datasets.CIFAR10(root='./data',train=False, transform=transform,download=True)



In [17]:
import torch.utils


train_loader = torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)


test_loader = torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=True,num_workers=2)

In [18]:
image ,label = train_data[0]



In [ ]:
image.size()

In [20]:
class_names = ['plane', 'car','bird','cat','deer','dog','frog','horse','ship','truck']



In [ ]:
class MyNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3,12,5)  #(12,28,28)
        self.pool = nn.MaxPool2d(2,2)  #(12,14,14)
        self.conv2 = nn.Conv2d(12,24,5) #(12,10,10) -->pool ->(12,5,5)
        self.fc1 = nn.Linear(24*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



In [ ]:


# Select device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define model and move to device
net = MyNetwork().to(device)  # Ensure the model is on the correct device

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training loop
for epoch in range(30):
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_loader):
        # Move data to device
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # Reset gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(images)

        # Calculate loss
        loss = loss_function(outputs, labels)
        loss.backward()

        # Update weights
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate average loss and accuracy
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    print(f"Epoch: {epoch + 1}/30  Loss: {epoch_loss:.4f}  Accuracy: {epoch_accuracy:.2f}%")


In [26]:
torch.save(net.state_dict(),'trained_net_pth')

In [ ]:
net = MyNetwork()
net.load_state_dict(torch.load('trained_net_pth'))

In [ ]:
correct = 0
total  = 0


net.eval()
for data in (test_loader):
    images,labels = data
    images,labels = images.to(device), labels.to(device)

    outputs = net(images)

    _,predicted = torch.max(outputs.data,1)
    correct+=(predicted==labels).sum().item()
    total+=labels.size(0)
accuracy = 100*correct/total
print(f"Accuracy of mynetwork is {accuracy}")